In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.cluster import SpectralClustering
import networkx.algorithms.community as nx_comm


In [2]:
meetings_meaningful_internal = pd.read_csv('meetings_meaningful_googledoc/meetings_meaningful_internal.csv')
df = meetings_meaningful_internal[meetings_meaningful_internal['Internal1'] & meetings_meaningful_internal['Internal2']].reset_index(drop=True)
df = df[['Source', 'Target', 'Weight']]

DG = nx.from_pandas_edgelist(df, source='Source', target='Target', edge_attr = 'Weight', create_using=nx.DiGraph())
orig_G = nx.Graph()
orig_G.add_edges_from(DG.edges(), weight=0)
for u,v,d in DG.edges(data=True):
    orig_G[u][v]['weight'] += d['Weight']
nodes = list(orig_G.nodes)

#Create a dictionary showing the index corresponds to each node_id
idx_vs_node_id = {k: v for v, k in enumerate(nodes)} #{1: 0, 4: 1, 6: 2, ...}

#Create the adjacency matrix
adj_mat = np.zeros((len(nodes), len(nodes)))

for r in range(len(df)):
    i = df.iloc[r]['Source']
    i = idx_vs_node_id[i]
    j = df.iloc[r]['Target']
    j = idx_vs_node_id[j]
    adj_mat[i][j] = 1
    adj_mat[j][i] = 1
    

In [3]:
#Spectral Clustering + Stopping rule
def first_two_eigen_valandvec(A):
    '''
    Return largest_eigenval in magnitude, largest_eigenvec,
    sec_largest_eigenval in magnitude, sec_largest_eigenvec
    '''
    w, v = np.linalg.eig(A) #eigenvalues and vectors
    w_abs = abs(w)

    largest_eigenval = w[0]; largest_idx = 0
    sec_largest_eigenval = w[1]; sec_largest_idx = 1
    if w_abs[1] > w_abs[0]:
        largest_eigenval = w[1]; largest_idx = 1
        sec_largest_eigenval = w[0]; sec_largest_idx = 0
    for i in np.arange(2, len(w)):
        if w_abs[i] > largest_eigenval:
            sec_largest_eigenval = largest_eigenval; sec_largest_idx = largest_idx;
            largest_eigenval = w[i]; largest_idx = i
        if largest_eigenval > w_abs[i] > sec_largest_eigenval:
            sec_largest_eigenval = w[i]; sec_largest_idx = i
    
    largest_eigenvec = v[largest_idx]
    sec_largest_eigenvec = v[sec_largest_idx]
    return largest_eigenval, largest_eigenvec, sec_largest_eigenval, sec_largest_eigenvec

def get_nonbt_mat(A):    
    d = A.sum(axis=0)
    D = np.zeros(A.shape)
    np.fill_diagonal(D, d)
    
    B = np.zeros(A.shape)
    I = np.zeros(A.shape)
    np.fill_diagonal(I, -1)
    
    B = np.concatenate((B, I), axis=0) #Left half of B_np
    tmp = np.concatenate((D-I, A), axis=0) #Right half of B_np
    B = np.concatenate((B, tmp), axis=1)
    
    return B

def nonbt_norm_approx(G):
    deg_list = list(dict(G.degree()).values())
    return np.sum(np.square(deg_list)) / np.sum(deg_list) - 1

def stopping_rule(A, G):
    '''
    Stop if returns True.
    '''
    B_nb = get_nonbt_mat(A)
    eigval1, _, eigval2, _ = first_two_eigen_valandvec(B_nb)
    threshold = np.sqrt(nonbt_norm_approx(orig_G))
    return not ((eigval1.real > threshold) & (eigval2.real > threshold))

def sub_network(A, G):
    G_nodes = list(G.nodes)
    c = SpectralClustering(n_clusters=2, affinity='precomputed').fit(A)
    c = c.labels_
    d = dict(zip(G_nodes, c))
    c = list(map(bool, c))
    
    subnodes1 = np.array(G_nodes)[c]
    #subedges1 = G.edges(subnodes1)
    neg_c = [not k for k in c]
    subnodes2 = np.array(G_nodes)[neg_c]
    #subedges2 = G.edges(subnodes2)
    
    #subG1 = G.subgraph(subnodes1)
    #subG2 = G.subgraph(subnodes2)
    return subnodes1, subnodes2, d

def spectral_clustering_hierarchical(A, G, d):
    if stopping_rule(A, G):
        return A, G, d

    subnodes1, subnodes2, res_d = sub_network(A, G)
    for key in res_d:
        d[key] = d[key] + str(res_d[key])

    subA1 = nx.to_numpy_matrix(G, nodelist = subnodes1)
    subA2 = nx.to_numpy_matrix(G, nodelist = subnodes2)
    #print(d)
    #left tree
    _, _, d = spectral_clustering_hierarchical(subA1, G.subgraph(subnodes1), d)
    #right tree
    _, _, d = spectral_clustering_hierarchical(subA2, G.subgraph(subnodes2), d)
    
    return A, G, d

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
empty_d = {k: '' for k in nodes}

all_runtime = []
result = pd.read_csv('meetings_meaningful_googledoc/meetings_meaningful_nodes_internal.csv')

for i in range(5):
    start_time = time.time()
    _, _, result_d = spectral_clustering_hierarchical(adj_mat, orig_G, empty_d)
    rt = time.time() - start_time
    all_runtime.append(rt)
    communities = list(set(result_d.values()))
    communities_rename = {k: v+1 for v, k in enumerate(communities)}
    result_communities = {k: communities_rename[v] for k,v in result_d.items()}
    lab = [result_communities[k] for k in result['id']]
    result['community'+str(i+1)] = lab

print(np.sum(all_runtime)/5)

4.714409494400025


In [6]:
result

,id,region,department,executive,leveldesignation,organization,groups,internal,community1,community2,community3,community4,community5
0,6,Central,Operations,Smith,Director,Procurement-Central,A,True,26,47,46,13,32
1,7,South,Operations,Smith,JuniorIC,Procurement-South,C,True,10,26,35,41,15
2,9,Central,Operations,Smith,Manager,OperationsCentral31,C,True,27,13,22,73,71
3,10,Central,R&D,Johnson,SeniorIC,ProductDesign-CA,C,True,58,36,45,58,20
4,14,Central,Operations,Smith,SeniorIC,ProductDesign-Central-G,C,True,44,23,32,51,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,1623,South,Operations,Smith,Manager,OperationsSouth132,C,True,28,54,19,34,62
348,1455,South,Operations,Johnson,Support,ProductTesting-South-C,C,True,44,23,32,51,66
349,1688,South,G&A,Brown,Support,HR-South,C,True,10,26,35,41,15
350,1589,South,Operations,Brown,Support,Logistics-South-A,C,True,34,32,67,55,54


In [7]:
result.to_csv("meetings_meaningful_googledoc/meetings_meaningful_internal_SCwSR.csv", index=False)